In [ ]:
import numpy as np

from scipy.interpolate import make_interp_spline
from scipy.interpolate import _bspl
import scipy.linalg as sl
import matplotlib.pyplot as plt

In [ ]:
def circle_vector(x,l=1,r=1):
    '''
    returns vector of nodes on circle
    '''
    assert len(x) > max(l, r)
    dx = np.diff(x)
    t = np.zeros(len(x) + l + r)
    t[:l] = [x[0] - sum(dx[-i:]) for i in range(l,0,-1)]
    t[l:-r] = x
    t[-r:] = [x[-1] + sum(dx[:i]) for i in range(1,r+1)]
    return t

In [ ]:
def B(x, k, i, t):
    if k == 0:
        if t[i] <= x < t[i+1]:
            return 1.0
        return 0.0
    if t[i+k] == t[i]:
        c1 = 0.0
    else:
        c1 = (x - t[i])/(t[i+k] - t[i]) * B(x, k-1, i, t)
    if t[i+k+1] == t[i+1]:
        c2 = 0.0
    else:
        c2 = (t[i+k+1] - x)/(t[i+k+1] - t[i+1]) * B(x, k-1, i+1, t)
    return c1 + c2

def bspline(x, t, c, k):
    n = len(t) - k - 1
    assert (n >= k+1) and (len(c) >= n)
    return sum(c[i] * B(x, k, i, t) for i in range(n))

def draw_spline(c, x, y, t, k):
    xxx = np.linspace(x[0],x[-1],len(x) * 40)
    plt.plot(xxx, np.vectorize(lambda x: bspline(x, t, c, k))(xxx))
    plt.scatter(x,y)
    plt.show()

In [ ]:
def make_interp_per_full_matr(x, y, t, k):
    '''
    ...
    
    Parameters
    ----------
    x : 1-D array, shape (n,)
        Values of x - coordinate of a given set of points.
    y : 1-D array, shape (n,)
        Values of y - coordinate of a given set of points.
    t : 1-D array, shape(n+2*k,)
        Vector of nodes.
    k : int
        The maximum degree of spline interpolant
        
    Returns
    -------
    c : 1-D array, shape (n+k-1,)
        Coefficients of B-spline interpolant
    '''
    # only odd 'k'
    assert k % 2 != 0 

    x, y, t = map(np.asarray, (x, y, t))

    n = x.size
    nt = t.size - k - 1

    # have `n` conditions for `nt` coefficients; need nt-n derivatives
    assert nt - n == k - 1

    # LHS: the collocation matrix + derivatives at edges
    A = np.zeros((n, nt), dtype=np.float_)

    # derivatives at x[0]:
    offset = 0

    # RHS
    y = np.r_[[0]*(k-1), y]

    # collocation matrix
    for j in range(n):
        xval = x[j]
        # find interval
        '''if xval == t[k]:
            left = k
        else:
            left = np.searchsorted(t, xval) - 1'''
        left = j + k

        # fill a row
        bb = _bspl.evaluate_all_bspl(t, k, xval, left)
        if (j == n - 1):
            A[j + offset, left-k:] = bb[:-1]
        else:
            A[j + offset, left-k:left+1] = bb

    res = np.zeros((n-1, n-1))
    res[:n-k, :] = np.copy(A[:n-k, : n-1]) # последние k-1 строки циклически сдвинутся
                                          # первые n-k - без изменений 
    for i in range(1, k):
      res[n-k-1+i, :i] = np.copy(A[n-k-1+i, -k:-k+i]) # заполнили строку левого блока (i последних элементов исходной строки)
      res[n-k-1+i, -k+i:] = np.copy(A[n-k-1+i, n-k-1+i:n-1]) # заполнили строку правой части (k - i первых элементов исходной строки)

    # переносим первые (k-1)/2 столбцов в конец матрицы
    res = res.transpose()
    res = np.concatenate((res[int((k-1)/2):, :], res[:int((k-1)/2), :]))
    res = res.transpose()
    return res

In [ ]:
def diagonal_form(a, k, upper = 1, lower= 1):
    """
    a is a numpy square matrix
    this function converts a square matrix to diagonal ordered form
    returned matrix in ab shape which can be used directly for scipy.linalg.solve_banded
    """
    n = a.shape[1]
    assert(np.all(a.shape ==(n,n)))
    
    ab = np.zeros((2*n-1, n))
    
    for i in range(n):
        ab[i,(n-1)-i:] = np.diagonal(a,(n-1)-i)
        
    for i in range(n-1): 
        ab[(2*n-2)-i,:i+1] = np.diagonal(a,i-(n-1))

    mid_row_inx = int(ab.shape[0]/2)
    upper_rows = [mid_row_inx - i for i in range(1, upper+1)]
    upper_rows.reverse()
    upper_rows.append(mid_row_inx)
    lower_rows = [mid_row_inx + i for i in range(1, lower+1)]
    keep_rows = upper_rows+lower_rows
    ab = ab[keep_rows,:]
    
    u = int((k-1)/2)
    ll, ur = np.zeros((u, u)), np.zeros((u, u))
    A_comp = np.zeros((k, n)) 

    ur[:, :] = np.copy(A[:u, -u:])
    ll[:, :] = np.copy(A[-u:, :u])

    return ab, ur, ll

In [ ]:
def woodbury(A, ll, ur, b, k):
  # A - banded matrix in compact form (shape (k, n-1))
  # ll - lower left block
  # ur - upper right block
  
  bs = int((k-1)/2)
  n = A.shape[1] + 1

  U = np.zeros((n-1, k-1))
  V = np.zeros((k-1, n-1)) # V transpose 
  
  # upper right

  U[:bs, :bs] = ur
  for j in range(bs): 
    V[j, -bs+j] = 1

  # lower left

  U[-bs:, -bs:] = ll
  for j in range(bs): 
    V[-bs+j, j] = 1
  
  Z = sl.solve_banded((bs, bs), A, U[:, 0]) # z0
  Z = np.expand_dims(Z, axis=0)
  
  for i in range(1, bs*2):
    zi = sl.solve_banded((bs, bs), A, U[:, i])
    zi = np.expand_dims(zi, axis=0)
    Z = np.concatenate((Z, zi), axis=0)

  Z = Z.transpose()
  H = sl.inv(np.identity(bs*2) + V.dot(Z))

  y = sl.solve_banded((bs, bs), A, b)
  c = y - Z.dot(H.dot(V.dot(y)))

  return c